# Assignment 8 Probabilistic Reasoning & Learning

### 8.1 EM Algorithm for binary matrix completion

#### a) Sanity Check 
Compute the mean popularity rating of each movie, given by the simple ratio $$\frac{\text{number of students who recommended the movie}}{\text{number of students who saw the movie}},$$ and sort the movies by this ratio. Print out the movie titles from least popular (I Feel Pretty) to most popular (Inception). Note how well these rankings do or do not corresponding to your individual preferences.

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = open('hw8_movies.txt').read().splitlines()
ids = open('hw8_ids.txt').read().splitlines()
ratings = np.loadtxt('hw8_ratings.txt', dtype='str')

In [22]:
movie_with_ratings = []
for i in range(len(movies)):
    rating = ratings[:,i]
    recommended = sum(rating == "1")
    seen = sum(rating != "?")
    movie_with_ratings.append((movies[i], recommended/seen))

df = pd.DataFrame(movie_with_ratings, columns =['Movie', 'Rating'])
df = df.sort_values(by="Rating")
# Print first 1-26, then 27-52, then 53-77
print(df[52:76])


                                           Movie    Rating
67                                         Joker  0.824000
25                                Les_Miserables  0.825000
26                                21_Jump_Street  0.825397
64                      Spiderman:_Far_From_Home  0.827160
2                                     Black_Swan  0.829787
70                                      Parasite  0.836364
20                                  The_Avengers  0.843648
69                                  The_Farewell  0.860465
23                              Django_Unchained  0.863946
31                                Now_You_See_Me  0.864979
62                             Avengers:_Endgame  0.868132
50                        Avengers:_Infinity_War  0.878571
28                           Wolf_of_Wall_Street  0.891667
65                                 The_Lion_King  0.894366
37                                     Gone_Girl  0.894410
5   Harry_Potter_and_the_Deathly_Hallows:_Part_1  0.9006

#### e) Implementation



In [23]:
probZ = np.loadtxt('hw8_probZ_init.txt', dtype='float32') 
probR = np.loadtxt('hw8_probR_init.txt', dtype='float32')

In [26]:
k = 4
n_iter = 256
T = len(ids)

In [29]:
prob_z = np.copy(probZ)
prob_r = np.copy(probR)
posteriors = np.empty([k,T], dtype='float32') 
log_likelihoods = []

In [43]:
def e_step(i, t, prob_z, prob_r):
    recommended, = np.where(ratings[t,:] == "1")
    not_recommended, = np.where(ratings[t,:] == "0")
    numerator = prob_z[i]*np.prod(prob_r[recommended,i])*np.prod(1-prob_r[not_recommended,i])
    denominator = 0
    for x in range(k):
        denominator += prob_z[x]*np.prod(prob_r[recommended,x])*np.prod(1-prob_r[not_recommended,x])
    return numerator/denominator

In [48]:
def m_step(i, j, posteriors, prob_r):
    seen, = np.where(ratings[:,j] == "1")
    not_seen, = np.where(ratings[:,j] == "?")
    return (np.sum(posteriors[i,seen])+prob_r[j,i]*np.sum(posteriors[i,not_seen]))/np.sum(posteriors[i,:])

In [49]:
def log_likelihood(t, prob_z, prob_r):
    sum = 0
    for i in range(k):
        recommended, = np.where(ratings[t,:] == "1")
        not_recommended, = np.where(ratings[t,:] == "0")
        sum += prob_z[i]*np.prod(prob_r[recommended,i])*np.prod(1-prob_r[not_recommended,i])
    return np.log(sum)

In [55]:
def em(prob_r, prob_z, log_likelihoods, posteriors, n_iter):
    for iter in range(n_iter+1):
        loglike = 0
        prob_z_temp = np.empty(k)
        prob_r_temp = np.empty([len(movies),k])
        for t in range(T):
            loglike += log_likelihood(t, prob_z, prob_r)
            for i in range(k):
                posteriors[i,t] = e_step(i, t, prob_z, prob_r)
        for i in range(k):
            prob_z_temp[i] = np.sum(posteriors[i,:])/T
            for j in range(len(movies)):
                prob_r_temp[j,i] = m_step(i, j, posteriors, prob_r)
        log_likelihoods.append(loglike/T)
        prob_z = prob_z_temp
        prob_r = prob_r_temp
    return log_likelihoods, posteriors, prob_r, prob_z

In [56]:
log_likelihoods, post, probr, probz = em(prob_r, prob_z, log_likelihoods, posteriors, n_iter)

In [57]:
print(f"Iteration: 0 \t Log: {round(log_likelihoods[0],4)}")
print(f"Iteration: 1 \t Log: {round(log_likelihoods[1],4)}")
print(f"Iteration: 2 \t Log: {round(log_likelihoods[2],4)}")
print(f"Iteration: 4 \t Log: {round(log_likelihoods[4],4)}")
print(f"Iteration: 8 \t Log: {round(log_likelihoods[8],4)}")
print(f"Iteration: 16 \t Log: {round(log_likelihoods[16],4)}")
print(f"Iteration: 32 \t Log: {round(log_likelihoods[32],4)}")
print(f"Iteration: 64 \t Log: {round(log_likelihoods[64],4)}")
print(f"Iteration: 128 \t Log: {round(log_likelihoods[128],4)}")
print(f"Iteration: 256 \t Log: {round(log_likelihoods[256],4)}")

Iteration: 0 	 Log: -27.0358
Iteration: 1 	 Log: -17.5604
Iteration: 2 	 Log: -16.0024
Iteration: 4 	 Log: -15.0606
Iteration: 8 	 Log: -14.5016
Iteration: 16 	 Log: -14.2638
Iteration: 32 	 Log: -14.1802
Iteration: 64 	 Log: -14.1701
Iteration: 128 	 Log: -14.164
Iteration: 256 	 Log: -14.1637


#### f) Personal movie recommendations

In [64]:
idx = ids.index("U09085243")
not_seen, = np.where(ratings[idx,:] == "?")
unseen_with_expected = []

for movie in not_seen:
    expected = 0
    for i in range(k):
        estep = e_step(i, idx, probz, probr)
        mstep = m_step(i, movie, post, probr)
        expected += estep*mstep
    unseen_with_expected.append((movies[movie], expected))

df2 = pd.DataFrame(unseen_with_expected, columns =['Unseen movie', 'Expected rating'])
df2 = df2.sort_values(by="Expected rating", ascending=False)



In [65]:
df2

,Unseen movie,Expected rating
17,The_Martian,0.957777
3,Captain_America:_The_First_Avenger,0.919661
15,Gone_Girl,0.917336
2,Thor,0.908369
29,The_Farewell,0.899042
23,Manchester_by_the_Sea,0.865086
21,Chappaquidick,0.860513
1,Toy_Story_3,0.850382
10,The_Perks_of_Being_a_Wallflower,0.833299
27,Darkest_Hour,0.832298
